##### <span style="color:#483D8B; font-weight:bold;">Import Libraries</span>

In [1]:
!pip install pyspellchecker
!pip install symspellpy
!pip install evaluate
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.4/158.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [2]:
import transformers
print(transformers.__version__)

4.52.4


In [3]:
#!pip uninstall preprocessing

In [4]:
import warnings

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import re
#import preprocessor as p
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from symspellpy.symspellpy import SymSpell, Verbosity
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import warnings
import torch

In [6]:
from datasets import Dataset
from transformers import TrainingArguments
from transformers import Trainer
import evaluate
from transformers import EarlyStoppingCallback #to stop training early when eval loss stops improving
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaTokenizerFast, RobertaConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support #metric functions

KeyboardInterrupt: 

##### <span style="color:#483D8B; font-weight:bold;">Import Dataset</span>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = r"C:\Users\User\Desktop\Ironhack_DA\Final-project-hate-speech-detection-with-NLP\Dataset-Hate-Speech-Detection.csv"

In [ ]:
path2 = "/content/drive/MyDrive/Dataset-Hate-Speech-Detection.csv"

In [ ]:
df = pd.read_csv(path2, low_memory=False)

In [ ]:
df

Classification:

- 0 - Hate Speech
- 1 - Offensive Language
- 2 - Neither/Neutral content

In [ ]:
df.shape

In [ ]:
df.reset_index(drop=True, inplace=True)
df.index = df.index + 0

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

NOTE: no null values & consistent datatypes

##### <span style="color:#483D8B; font-weight:bold;">Cleaning tweet column</span>

In [ ]:
from IPython.display import display
pd.set_option('display.max_colwidth', None)

In [ ]:
display(df.iloc[:10, :2])

In [ ]:
df[df['class'] == 0]

In [ ]:
df.groupby('class').nunique()

In [ ]:
class_count = df['class'].value_counts()

ax = class_count.plot(kind='bar')

for i, value in enumerate(class_count):
    ax.text(i, value + 0.5, str(value), ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
display(df.iloc[:5, :2])

In [ ]:
nltk.download('stopwords')

In [ ]:
max_edit_distance_dictionary = 2
prefix_length = 7
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)

dictionary_path = "frequency_dictionary_en_82_765.txt"  # path to the downloaded dictionary
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

#p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.EMOJI)

stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

def correct_spelling_symspell(text):
    corrected_words = []
    for word in text.split():
        # Skip short words and stopwords to speed up
        if len(word) < 3 or word in stop_words:
            corrected_words.append(word)
            continue

        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            corrected_words.append(suggestions[0].term)
        else:
            corrected_words.append(word)
    return " ".join(corrected_words)

def clean_tweet(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags (optional: keep the text if you want)
    text = re.sub(r'#\w+', '', text)
    # Remove emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+",
        flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text).strip()
    text = correct_spelling_symspell(text)
    pattern = r'([{}])\1+'.format(re.escape(punctuation))
    text = re.sub(pattern, r'\1', text)
    tokens = tokenizer.tokenize(text)
    words = [w for w in tokens if w not in stop_words]
    return " ".join(words)

df['tweet_cleaned'] = df['tweet'].apply(clean_tweet)

In [ ]:
df['tweet_cleaned'] = df['tweet_cleaned'].str.replace(r'!\s*rt\s*:', '', regex=True).str.replace(r"[^\w\s.,!?']", "", regex=True)

In [ ]:
df

- Word cloud, to find common words in each class
- NLP techniques to get insights

##### <span style="color:#483D8B; font-weight:bold;">Model bulding</span>

##### <span style="color:#483D8B; font-weight:bold;">1 - Handling Imbalanced Data</span>

In [ ]:
df.groupby('class').nunique()

In [ ]:
df.drop('tweet', inplace = True, axis=1)

In [ ]:
df = df.drop_duplicates(subset='tweet_cleaned', keep='first').reset_index(drop=True)

In [ ]:
multi_class_tweets = df.groupby('tweet_cleaned')['class'].nunique()

tweets_with_multiple_classes = multi_class_tweets[multi_class_tweets > 1]

print(tweets_with_multiple_classes)

In [ ]:
df_0 = df[df['class'] == 0]   # All hate speech tweets
df_1 = df[df['class'] == 1]   # All offensive tweets
df_2 = df[df['class'] == 2]   # All neutral tweets

In [ ]:
max_size = df_1.shape[0]
max_size

#there are duplicate rows

In [ ]:
df.groupby('class').nunique()

In [ ]:
df_0_upsampled = resample(df_0, replace=True, n_samples=max_size, random_state=42)
df_2_upsampled = resample(df_2, replace=True, n_samples=max_size, random_state=42)

In [ ]:
df_balanced = pd.concat([df_0_upsampled, df_2_upsampled, df_1])

In [ ]:
df_balanced.groupby('class').count()

In [ ]:
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df_balanced.groupby('class').count()

In [ ]:
df_balanced.head(2)

In [ ]:
df_balanced["char_length"] = df_balanced["tweet_cleaned"].apply(len)
print(df_balanced["char_length"].describe())

##### <span style="color:#483D8B; font-weight:bold;">2 - RoBERTa model bulding</span>

Using a lighter version or Roberta, distilroberta-base

In [ ]:
df_reduced = df_balanced.groupby('class', group_keys=False).apply(lambda x: x.sample(frac=0.8, random_state=42)).reset_index(drop=True) #changed from 0.7 to 0.8

In [ ]:
df_reduced = df_reduced.drop(columns=['char_length'])

In [ ]:
df_reduced.groupby('class').count()

In [ ]:
X = df_reduced["tweet_cleaned"]
y = df_reduced["class"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("distilroberta-base")

In [ ]:
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=150) # changed the length (actuals length 147)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=150)

In [ ]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.reset_index(drop=True)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, y_train)
test_dataset = TweetDataset(test_encodings, y_test)

In [ ]:
config = RobertaConfig.from_pretrained(
        "distilroberta-base",
        num_labels=3,
        hidden_dropout_prob=0.3,                  # Increase dropout to help prevent overfitting
        attention_probs_dropout_prob=0.3          # Dropout in attention layers too
)

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("distilroberta-base", config=config)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',  # Where to save model outputs and checkpoints
    num_train_epochs=10,     # Train up to 10 epochs (early stopping may stop earlier)
    per_device_train_batch_size=20, # Number of samples per batch for training
    per_device_eval_batch_size=20, # Number of samples per batch for evaluation
    logging_dir='./logs',          # Directory for training logs
    logging_steps=1000,             # Log training info every 1000 steps
    # no_cuda=True  # to force CPU usage if needed
    eval_strategy="epoch",              # Run evaluation after every epoch to check model performance
    save_strategy="epoch",                    # Save model checkpoint after each epoch
    load_best_model_at_end=True,              # After training, load the checkpoint with best evaluation loss
    metric_for_best_model="eval_loss",        # Use evaluation loss to determine which checkpoint is best
    weight_decay=0.01                         # Add weight decay (L2 regularization) to prevent overfitting
)

In [ ]:
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2                 # Stop training if eval loss does not improve for 2 consecutive epochs
)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,          # function for evaluating metrics
    callbacks=[early_stopping],               # Enable early stopping during training
)

In [ ]:
trainer.train()

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset,  # only evaluation dataset here
)

eval_results = trainer.evaluate()  # evaluates the model on test_dataset
print(eval_results)

The model is randomly guessing the correct answer, eval_loss: 1.1